In [1]:
import YXJ
import torch
from torch.utils.data import TensorDataset, DataLoader
import pandas as pd
import numpy as np
import torch.nn.functional as F

In [2]:
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np

class TimeSeriesDataset(Dataset):
    def __init__(self, csv_file, steps_per_sample):
        self.data_frame = pd.read_csv(csv_file) # 读取CSV文件
        self.steps_per_sample = steps_per_sample
        # 预处理数据
        n_samples = len(self.data_frame) // steps_per_sample
        self.data_frame = self.data_frame.iloc[:n_samples * steps_per_sample]
        self.data_frame = self.data_frame.values.reshape(n_samples, steps_per_sample,-1)
        
    def __len__(self):
        return len(self.data_frame)
    
    def __getitem__(self, idx):
        sample = self.data_frame[idx]
        X = sample[:, :-1]  # 所有行，除了最后一列
        y = sample[0, -1]   # 第一个时间步的标签
        
        # 转换为Tensor
        X = torch.tensor(X, dtype=torch.float32)
        y = torch.tensor(y, dtype=torch.float32)
        return X, y


# 使用示例
SAMPLES_PER_GESTURE = 100
dataset = TimeSeriesDataset('normalized_data.csv', SAMPLES_PER_GESTURE)
GESTURES = ["smoke", "nosmoke"]  # 手势列表
NUM_GESTURES= len(GESTURES);

# 分割数据集
from torch.utils.data import random_split
dataset_size = len(dataset)
train_size = int(dataset_size * 0.7)  # 70% 数据用于训练
valid_size = int(dataset_size * 0.2)  # 20% 数据用于验证
test_size = dataset_size - train_size - valid_size  # 剩余10% 用于测试
batch_size =64;
train_dataset, valid_dataset, test_dataset = random_split(dataset, [train_size, valid_size, test_size])
train_loader = DataLoader(train_dataset, batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size, shuffle=False)

#YXJ.check_dataloader(test_loader)

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim

# 定义模型
class IMULSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super(IMULSTM, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers

        # 定义LSTM层
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)

        # 定义输出层
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        # 初始化隐藏状态和细胞状态
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).to(x.device)

        # 从LSTM层获取输出
        out, _ = self.lstm(x, (h0, c0))

        # 只取LSTM输出的最后一步
        out = self.fc(out[:, -1, :])
        out = out.squeeze(-1)  # 使用 squeeze(-1) 仅尝试移除最后一个维度，如果它的大小为1
        return out

In [4]:
def validate(model, device, criterion, test_loader):
    model.eval()
    validation_loss = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            validation_loss += criterion(output, target).item()
    validation_loss /= len(test_loader)  # 计算平均验证损失
    return validation_loss
    
def train(model, device, train_loader, criterion, optimizer, epochs):
    model.train()
    train_losses = []  # 用于存储每个epoch的损失值
    for epoch in range(epochs):
        for data, target in train_loader:
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()           # 清零梯度
            output = model(data)            # 前向传播
            loss = criterion(output, target) # 计算损失
            loss.backward()                 # 反向传播
            optimizer.step()                # 更新参数
        train_losses.append(loss.item())
        print(f'Epoch {epoch+1}, Loss: {loss.item()}')
    return train_losses
    
def train_vali(model, device, train_loader, test_loader, criterion, optimizer, epochs):
    model.train()
    train_losses = []
    validation_losses = []
    for epoch in range(epochs):
        for data, target in train_loader:
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()  # 清零梯度
            output = model(data)  # 前向传播
            loss = criterion(output, target)  # 计算损失
            loss.backward()  # 反向传播
            optimizer.step()  # 更新参数
        train_losses.append(loss.item())
        
        # 在每个epoch后计算验证集上的损失
        validation_loss = validate(model, device, criterion, test_loader)
        validation_losses.append(validation_loss)
        
        print(f'Epoch {epoch+1}, Training Loss: {loss.item()}, Validation Loss: {validation_loss}')

    return train_losses, validation_losses
    
def test(model, device, test_loader):
    model.eval()  # 将模型设置为评估模式
    test_loss = 0
    correct = 0  # 用于累计正确预测的数量
    with torch.no_grad():  # 在评估阶段不计算梯度
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.cross_entropy(output, target, reduction='sum').item()  # 累加损失值
            pred = output.argmax(dim=1, keepdim=True)  # 找到概率最高的类别作为预测
            correct += pred.eq(target.view_as(pred)).sum().item()  # 如果预测正确，则累加到correct变量中

    test_loss /= len(test_loader.dataset)  # 计算平均损失
    accuracy = 100. * correct / len(test_loader.dataset)  # 计算准确率

    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({accuracy:.0f}%)')
    print(len(test_loader.dataset))

In [5]:
# 超参数设置
input_dim = 9   # 输入数据维度 (例如: IMU和GPS数据)
hidden_dim = 128  # LSTM隐藏层的维度
num_layers = 2  # LSTM的层数
output_dim = 1  # 输出的维度 (根据你的任务调整，例如分类数或回归问题的维度)

# 假设我们有一些随机生成的数据作为示例
# 输入数据的形状: (batch_size, sequence_length, input_dim)


# 使用GPU
use_cuda = torch.cuda.is_available()
device=torch.device('cuda' if use_cuda else 'cpu')

model =IMULSTM(input_dim, hidden_dim, num_layers, output_dim).to(device)
# 定义损失函数和优化器
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

train_losses, validation_losses=train_vali(model, device, train_loader, valid_loader, criterion, optimizer, epochs=10)
YXJ.draw_loss(train_losses,validation_losses)
test(model, device, valid_loader)
test(model, device, test_loader)

Epoch 1, Training Loss: 0.15063068270683289, Validation Loss: 0.08689358395834763
Epoch 2, Training Loss: 0.05140874907374382, Validation Loss: 0.07311532563633388
Epoch 3, Training Loss: 0.012624135240912437, Validation Loss: 0.0669750259257853
Epoch 4, Training Loss: 0.11888856440782547, Validation Loss: 0.09074888564646244
Epoch 5, Training Loss: 0.13665825128555298, Validation Loss: 0.13891812331146663
Epoch 6, Training Loss: 0.09512221813201904, Validation Loss: 0.10590417672776514
Epoch 7, Training Loss: 0.2491966187953949, Validation Loss: 0.24419866046971744


KeyboardInterrupt: 